In [1]:
# Selenium imports
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

import regex as re
import requests
from bs4 import BeautifulSoup

C:\Users\Felipe\AppData\Local\Temp\ipykernel_10884\599045925.py:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [ ]:
# General Helpers

def price_multiple_replace(text, dict=None):
    """
    Multiple regex replacements (to replace in key, replacement in value) 
    """
    if not dict:
        dict = {"," : ".", "." : ""}
        
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))
    
    # For each match, look-up corresponding value in dictionary
    return float(regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text))

# Zonaprop

In [ ]:
url = 'https://www.zonaprop.com.ar/inmuebles-alquiler.html'

In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 47.4MB/s]
C:\Users\Felipe\AppData\Local\Temp\ipykernel_13632\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [8]:
driver.get(url)

# Argenprop

In [17]:
# ARGENPROP WORKING CORRECTLY

headers = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "http://thewebsite.com",
    "Connection": "keep-alive"}

url_argenprop = 'https://www.argenprop.com/departamento-y-casa-venta-y-alquiler-y-alquiler-temporal'


result = requests.get(url_argenprop, headers=headers)

soup = BeautifulSoup(result.text, 'lxml')

# Links
base_link = 'https://www.argenprop.com'
div_container = soup.find_all('div',{'class':'listing__item'})
links = []
for div in div_container:
    a_link = div.find_all('a', href=True)
    last_link = a_link[0]['href']
    links.append(base_link+last_link)

# City
city = soup.find_all('p', {'class':'card__title--primary show-mobile'})
city = [c.text for c in city]


# Property Type
property_type = soup.find_all('p', {'class':'card__title--primary hide-mobile'})
property_type = [c.text.split(' ')[0] for c in property_type]

# Contract type
contract_type = soup.find_all('p', {'class':'card__title--primary hide-mobile'})
contract_type = [c.text.split(' ')[2] for c in contract_type]


# Rooms
ul_container = soup.find_all('ul', {'class':'card__main-features'})
rooms = []
for ul in ul_container:
    amb = re.findall('(\d+)\sambiente', str(ul.find_all('span')))
    if len(amb)>0:
        rooms.append(int(amb[0]))
    else:
        rooms.append(1)
        
# Currency type
currency = soup.find_all('span', {'class':'card__currency'})
currency = [c.text for c in currency]


# Price
buy_rent_currency = []
buy_rent_price = []

expensas_currency = []
expensas_price = []

all_currency_and_price = soup.find_all('p', {'class':'card__price'})
all_currency_and_price = [c.text for c in all_currency_and_price]
all_currency_and_price = [re.findall('(USD|\$)\s?(\d+(?:[,.]\d+)*)', str(c)) for c in all_currency_and_price]

for cp in all_currency_and_price:
    if len(cp)>0:
        buy_rent_currency.append(cp[0][0])
        buy_rent_price.append(price_multiple_replace(cp[0][1]))
        if len(cp)>1:
            expensas_currency.append(cp[1][0])
            expensas_price.append(price_multiple_replace(cp[1][1]))
    else:
        buy_rent_currency.append('consultar_precio')
        buy_rent_price.append(0)
        expensas_currency.append('consultar_precio')
        expensas_price.append(0)

# Adress
address = soup.find_all('h2', {'class':'card__address'})
address = [c.text.strip() for c in address]
address = [re.findall('.+', str(c))[0] for c in address]

# Mercado Libre

In [ ]:
# Helpers
def get_expensas_price(links):
    
    """
    For Mercado Libre
    """
    expensas_price = []

    link_expensas = [l for l in links if 'departamento' in l]

    for l_expensas in link_expensas:
        result = requests.get(l_expensas, headers=headers)
        soup = BeautifulSoup(result.text, 'lxml')
        try:
            exp = soup.find('p', {'class':'ui-pdp-color--GRAY ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-maintenance-fee-ltr'}).text
        except:
            exp = '0'
        expensas_price.append(exp)

    expensas_price = [re.findall('\d+(?:[,.]\d+)*', ex)[0] for ex in expensas_price]
    expensas_price = [int(price_multiple_replace(c)) for c in expensas_price]
    
    return expensas_price

In [132]:
headers = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "http://thewebsite.com",
    "Connection": "keep-alive"}

url_argenprop = 'https://inmuebles.mercadolibre.com.ar/departamentos/'


result = requests.get(url_argenprop, headers=headers)

soup = BeautifulSoup(result.text, 'lxml')

In [133]:
# Links
div_container = soup.find_all('div',{'class':'ui-search-result__wrapper shops__result-wrapper'})
links = []
for div in div_container:
    a_link = div.find_all('a', href=True)
    link = a_link[0]['href']
    links.append(link)

# City
city = soup.find_all('span', {'class':'ui-search-item__group__element ui-search-item__location shops__items-group-details'})
city = [c.text for c in city]

# Adress
address = soup.find_all('span', {'class':'ui-search-item__group__element ui-search-item__location shops__items-group-details'})
address = [c.text.split(',')[0] for c in address]

# Property type
property_type = soup.find_all('span', {'class':'ui-search-item__group__element ui-search-item__subtitle shops__items-group-details'})
property_type = [c.text.split(' en ')[0] for c in property_type]

# Contract type
contract_type = soup.find_all('span', {'class':'ui-search-item__group__element ui-search-item__subtitle shops__items-group-details'})
contract_type = [c.text.split(' en ')[-1] for c in contract_type]

# Rooms
rooms = soup.find_all('li', {'class':'ui-search-card-attributes__attribute'})
rooms = [int(c.text.split(' ')[0]) for c in rooms if 'm²' not in c.text]

# All Currency type
buy_rent_currency = soup.find_all('span', {'class':'price-tag-symbol'})
buy_rent_currency = [c.text.replace('U$S', 'USD') for c in buy_rent_currency]
buy_rent_currency = buy_rent_currency[len(buy_rent_currency)-len(links):] # Eliminate those with no link
expensas_currency = ['$']*len(links)

# Buy Rent Price
buy_rent_price = []
buy_rent_price = soup.find_all('span', {'class':'price-tag-fraction'})
buy_rent_price = [c.text for c in buy_rent_price]
buy_rent_price = [int(price_multiple_replace(c)) for c in buy_rent_price]
buy_rent_price = buy_rent_price[len(buy_rent_price)-len(links):] # Eliminate those with no link

# Expensas Price
scrap_expensas_ML = False
if scrap_expensas_ML:
   expensas_price = get_expensas_price(links)
else:
    expensas_price = [0]*len(links)